In [143]:
import re

<link href="style.css" rel="stylesheet"></link>

<article class="day-desc"><h2>--- Day 8: Haunted Wasteland ---</h2><p>You're still riding a camel across Desert Island when you spot a sandstorm quickly approaching. When you turn to warn the Elf, she disappears before your eyes! To be fair, she had just finished warning you about <em>ghosts</em> a few minutes ago.</p>
<p>One of the camel's pouches is labeled "maps" - sure enough, it's full of documents (your puzzle input) about how to navigate the desert. At least, you're pretty sure that's what they are; one of the documents contains a list of left/right instructions, and the rest of the documents seem to describe some kind of <em>network</em> of labeled nodes.</p>
<p>It seems like you're meant to use the <em>left/right</em> instructions to <em>navigate the network</em>. Perhaps if you have the camel follow the same instructions, you can escape the haunted wasteland!</p>
<p>After examining the maps for a bit, two nodes stick out: <code>AAA</code> and <code>ZZZ</code>. You feel like <code>AAA</code> is where you are now, and you have to follow the left/right instructions until you reach <code>ZZZ</code>.</p>
<p>This format defines each <em>node</em> of the network individually. For example:</p>
<pre><code>RL

AAA = (BBB, CCC)
BBB = (DDD, EEE)
CCC = (ZZZ, GGG)
DDD = (DDD, DDD)
EEE = (EEE, EEE)
GGG = (GGG, GGG)
ZZZ = (ZZZ, ZZZ)
</code></pre>

<p>Starting with <code>AAA</code>, you need to <em>look up the next element</em> based on the next left/right instruction in your input. In this example, start with <code>AAA</code> and go <em>right</em> (<code>R</code>) by choosing the right element of <code>AAA</code>, <code><em>CCC</em></code>. Then, <code>L</code> means to choose the <em>left</em> element of <code>CCC</code>, <code><em>ZZZ</em></code>. By following the left/right instructions, you reach <code>ZZZ</code> in <code><em>2</em></code> steps.</p>
<p>Of course, you might not find <code>ZZZ</code> right away. If you run out of left/right instructions, repeat the whole sequence of instructions as necessary: <code>RL</code> really means <code>RLRLRLRLRLRLRLRL...</code> and so on. For example, here is a situation that takes <code><em>6</em></code> steps to reach <code>ZZZ</code>:</p>
<pre><code>LLR

AAA = (BBB, BBB)
BBB = (AAA, ZZZ)
ZZZ = (ZZZ, ZZZ)
</code></pre>

<p>Starting at <code>AAA</code>, follow the left/right instructions. <em>How many steps are required to reach <code>ZZZ</code>?</em></p>
</article>


In [144]:
case_1 = """
RL

AAA = (BBB, CCC)
BBB = (DDD, EEE)
CCC = (ZZZ, GGG)
DDD = (DDD, DDD)
EEE = (EEE, EEE)
GGG = (GGG, GGG)
ZZZ = (ZZZ, ZZZ)
"""

In [145]:
case_2 = """
LLR

AAA = (BBB, BBB)
BBB = (AAA, ZZZ)
ZZZ = (ZZZ, ZZZ)

"""

In [146]:
from itertools import cycle


def count_steps(s: str) -> int:
    s = s.strip()
    instructions, nodes = re.split(r"\s+\n", s)
    instructions = instructions.strip()

    nodes = nodes.splitlines()
    nodes = [re.findall(r"[A-Z]{3}", node) for node in nodes]
    graph = {node: (left, right) for node, left, right in nodes}

    current = "AAA"
    goal = "ZZZ"

    for i, instruction in enumerate(cycle(instructions), start=1):
        current = graph[current][0 if instruction == "L" else 1]
        if current == goal:
            return i


assert count_steps(case_1) == 2
assert count_steps(case_2) == 6

In [147]:
with open("./input/day8.txt") as f:
    print(count_steps(f.read()))

24253


In [148]:
def count_cache(s: str) -> int:
    s = s.strip()
    instructions, nodes = re.split(r"\s+\n", s)
    instructions = instructions.strip()

    nodes = nodes.splitlines()
    nodes = [re.findall(r"[A-Z]{3}", node) for node in nodes]
    graph = {node: (left, right) for node, left, right in nodes}

    goal = "ZZZ"

    paths_containing_goal = {}
    full_cylce_graph = {}
    for node in graph.keys():
        current = node
        for step, inst in enumerate(instructions, start=1):
            current = graph[current][0 if inst == "L" else 1]
            if current == goal:
                paths_containing_goal[node] = step
        full_cylce_graph[node] = current

    current = "AAA"
    steps = 0

    while current not in paths_containing_goal:
        steps += len(instructions)
        current = full_cylce_graph[current]

    return steps + paths_containing_goal[current]


assert count_cache(case_1) == 2
assert count_cache(case_2) == 6

In [149]:
with open("./input/day8.txt") as f:
    print(count_cache(f.read()))

24253


<link href="style.css" rel="stylesheet"></link>

<main>

<p>Your puzzle answer was <code>24253</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>The <span title="Duhduhduhduhduh! Dah, duhduhduhduhduh!">sandstorm</span> is upon you and you aren't any closer to escaping the wasteland. You had the camel follow the instructions, but you've barely left your starting position. It's going to take <em>significantly more steps</em> to escape!</p>
<p>What if the map isn't for people - what if the map is for <em>ghosts</em>? Are ghosts even bound by the laws of spacetime? Only one way to find out.</p>
<p>After examining the maps a bit longer, your attention is drawn to a curious fact: the number of nodes with names ending in <code>A</code> is equal to the number ending in <code>Z</code>! If you were a ghost, you'd probably just <em>start at every node that ends with <code>A</code></em> and follow all of the paths at the same time until they all simultaneously end up at nodes that end with <code>Z</code>.</p>
<p>For example:</p>
<pre><code>LR

11A = (11B, XXX)
11B = (XXX, 11Z)
11Z = (11B, XXX)
22A = (22B, XXX)
22B = (22C, 22C)
22C = (22Z, 22Z)
22Z = (22B, 22B)
XXX = (XXX, XXX)
</code></pre>

<p>Here, there are two starting nodes, <code>11A</code> and <code>22A</code> (because they both end with <code>A</code>). As you follow each left/right instruction, use that instruction to <em>simultaneously</em> navigate away from both nodes you're currently on. Repeat this process until <em>all</em> of the nodes you're currently on end with <code>Z</code>. (If only some of the nodes you're on end with <code>Z</code>, they act like any other node and you continue as normal.) In this example, you would proceed as follows:</p>
<ul>
<li>Step 0: You are at <code>11A</code> and <code>22A</code>.</li>
<li>Step 1: You choose all of the <em>left</em> paths, leading you to <code>11B</code> and <code>22B</code>.</li>
<li>Step 2: You choose all of the <em>right</em> paths, leading you to <code><em>11Z</em></code> and <code>22C</code>.</li>
<li>Step 3: You choose all of the <em>left</em> paths, leading you to <code>11B</code> and <code><em>22Z</em></code>.</li>
<li>Step 4: You choose all of the <em>right</em> paths, leading you to <code><em>11Z</em></code> and <code>22B</code>.</li>
<li>Step 5: You choose all of the <em>left</em> paths, leading you to <code>11B</code> and <code>22C</code>.</li>
<li>Step 6: You choose all of the <em>right</em> paths, leading you to <code><em>11Z</em></code> and <code><em>22Z</em></code>.</li>
</ul>
<p>So, in this example, you end up entirely on nodes that end in <code>Z</code> after <code><em>6</em></code> steps.</p>
<p>Simultaneously start on every node that ends with <code>A</code>. <em>How many steps does it take before you're only on nodes that end with <code>Z</code>?</em></p>
</article>


In [150]:
case_ghosty = """
LR

11A = (11B, XXX)
11B = (XXX, 11Z)
11Z = (11B, XXX)
22A = (22B, XXX)
22B = (22C, 22C)
22C = (22Z, 22Z)
22Z = (22B, 22B)
XXX = (XXX, XXX)
"""

In [151]:
def ghosty_too_slow(s: str) -> int:
    s = s.strip()
    instructions, nodes = re.split(r"\s+\n", s)
    instructions = instructions.strip()

    nodes = nodes.splitlines()
    nodes = [re.findall(r"[A-Z0-9]{3}", node) for node in nodes]
    graph = {node: (left, right) for node, left, right in nodes}

    current = [node for node, _, _ in nodes if node[-1] == "A"]

    for i, instruction in enumerate(cycle(instructions), start=1):
        direction = 0 if instruction == "L" else 1
        current = [graph[c][direction] for c in current]
        if all(c[-1] == "Z" for c in current):
            return i


assert ghosty_too_slow(case_ghosty) == 6

In [152]:
# with open("./input/day8.txt") as f:
#     print(ghosty_too_slow(f.read()))

In [163]:
from itertools import cycle
from math import lcm


def ghosty(s: str) -> int:
    s = s.strip()
    instructions, nodes = re.split(r"\s+\n", s)
    instructions = instructions.strip()

    nodes = nodes.splitlines()
    nodes = [re.findall(r"[A-Z0-9]{3}", node) for node in nodes]
    graph = {node: (left, right) for node, left, right in nodes}

    currents = [node for node in graph.keys() if node[-1] == "A"]

    steps_container = []
    for current in currents:
        for i, instruction in enumerate(cycle(instructions), start=1):
            current = graph[current][0 if instruction == "L" else 1]
            if current[-1] == "Z":
                steps_container.append(i)
                break

    return lcm(*steps_container)  # only works because of cycles


print(ghosty(case_ghosty))

6


In [164]:
with open("./input/day8.txt") as f:
    print(ghosty(f.read()))

12357789728873


In [159]:
12357789728873 // 307

40253386739

<link href="style.css" rel="stylesheet"></link>

<main>

<p>Your puzzle answer was <code>12357789728873</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>
<p>At this point, you should <a href="/2023">return to your Advent calendar</a> and try another puzzle.</p>
<p>If you still want to see it, you can <a href="8/input" target="_blank">get your puzzle input</a>.</p>
<p>You can also <span class="share">[Share<span class="share-content">on
  <a href="https://twitter.com/intent/tweet?text=I%27ve+completed+%22Haunted+Wasteland%22+%2D+Day+8+%2D+Advent+of+Code+2023&amp;url=https%3A%2F%2Fadventofcode%2Ecom%2F2023%2Fday%2F8&amp;related=ericwastl&amp;hashtags=AdventOfCode" target="_blank">Twitter</a>
  <a href="javascript:void(0);" onclick="var ms; try{ms=localStorage.getItem('mastodon.server')}finally{} if(typeof ms!=='string')ms=''; ms=prompt('Mastodon Server?',ms); if(typeof ms==='string' &amp;&amp; ms.length){this.href='https://'+ms+'/share?text=I%27ve+completed+%22Haunted+Wasteland%22+%2D+Day+8+%2D+Advent+of+Code+2023+%23AdventOfCode+https%3A%2F%2Fadventofcode%2Ecom%2F2023%2Fday%2F8';try{localStorage.setItem('mastodon.server',ms);}finally{}}else{return false;}" target="_blank">Mastodon</a></span>]</span> this puzzle.</p>
</main>
